## Fake News Classification using Bi-Directional RNN

In [1]:
import pandas as pd 
import numpy as np 
df = pd.read_csv(r"D:\Python DataScience\Fake News Dataset\train.csv")

In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Dropping null values 
df = df.dropna()
X= df.drop('label',axis=1)
y = df['label']

In [4]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [5]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [6]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [7]:
X.shape,y.shape

((18285, 4), (18285,))

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense,Bidirectional,Dropout

In [9]:
### Vocabulary size
## require when you do one hot encoding 

voc_size=5000

## Onehot Representation

In [11]:
messages=X.copy()
print(messages['title'][1])
messages.reset_index(inplace=True)

FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart


In [13]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harshvardhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:5]

[[813, 320, 1422, 224, 3366, 1907, 2651, 3554, 3822, 3883],
 [2709, 4445, 1282, 1044, 3072, 3376, 222],
 [2973, 2026, 2124, 4176],
 [755, 3426, 1675, 3240, 2395, 3249],
 [996, 3072, 3281, 3909, 4460, 4409, 3072, 1940, 3722, 571]]

## Embedding Representation 


In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs[:5])

[[   0    0    0    0    0    0    0    0    0    0  813  320 1422  224
  3366 1907 2651 3554 3822 3883]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 2709
  4445 1282 1044 3072 3376  222]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2973 2026 2124 4176]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
   755 3426 1675 3240 2395 3249]
 [   0    0    0    0    0    0    0    0    0    0  996 3072 3281 3909
  4460 4409 3072 1940 3722  571]]


In [20]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [24]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 26s 2ms/sample - loss: 0.3034 - accuracy: 0.8549 - val_loss: 0.2006 - val_accuracy: 0.9158
Epoch 2/10
12250/12250 [==============================] - 4s 356us/sample - loss: 0.1329 - accuracy: 0.9503 - val_loss: 0.2055 - val_accuracy: 0.9176
Epoch 3/10
12250/12250 [==============================] - 4s 356us/sample - loss: 0.0909 - accuracy: 0.9669 - val_loss: 0.2525 - val_accuracy: 0.9127
Epoch 4/10
12250/12250 [==============================] - 4s 346us/sample - loss: 0.0682 - accuracy: 0.9758 - val_loss: 0.2696 - val_accuracy: 0.9027
Epoch 5/10
12250/12250 [==============================] - 4s 347us/sample - loss: 0.0461 - accuracy: 0.9839 - val_loss: 0.3595 - val_accuracy: 0.8961
Epoch 6/10
12250/12250 [==============================] - 4s 344us/sample - loss: 0.0303 - accuracy: 0.9906 - val_loss: 0.3992 - val_accuracy: 0.9089
Epoch 7/10
12250/12250 [============================

In [25]:
y_pred1=model1.predict_classes(X_test)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred1))
print(accuracy_score(y_test,y_pred1))

[[3085  334]
 [ 229 2387]]
0.9067108533554267


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

